In [ ]:
import masterjdv

Estimation des temps de restitution pour 10 x 10 100 x 100 et 1000 x 1000

In [ ]:
grid = masterjdv.init_grid(10)
%timeit masterjdv.evolution1(grid)

In [ ]:
grid = masterjdv.init_grid(100)
%timeit masterjdv.evolution1(grid)

In [ ]:
grid = masterjdv.init_grid(1000)
%timeit masterjdv.evolution1(grid)

On constate un temps d'exécution qui croit linéairement en fonction de la taille du problème (nombre de cellule  x100 à chaque fois) ce qui est cohérent avec notre algorithme qui applique un même calcul pour chaque cellule de la grille.

# Profilage

Définition d'une grille 2000 x 2000

In [ ]:
grid = masterjdv.init_grid(2000)

Profil de evolution1

In [ ]:
import cProfile
#cProfile.run("masterjdv.evolution1(grid)")
%prun -D evol1.prof masterjdv.evolution1(grid)

# Inlining
Définition de la fonction evolution1_corr dans laquelle on intègre le calcul de nb_neigh (inlining)

In [ ]:
def evolution1_corr(grid):
    irange=grid.shape[0]
    jrange=grid.shape[1]
    res_grid=[[] for i in range(irange)]
    for j in range(jrange):
        for i in range(irange):
            # by default cell_state is dead
            cell_state=0
            # loop over neighs to count living cells
            biinf=max(0,i-1)
            bisup=min(irange-1,i+1)
            bjinf=max(0,j-1)
            bjsup=min(jrange-1,j+1)
            nb_neigh=0
            # loop over neighs to count living cells
            for ii in range(biinf,bisup+1):
                for jj in range(bjinf,bjsup+1):
                    if grid[ii][jj] == 1:
                        nb_neigh=nb_neigh+1
            # in my count I count current cell itself substract the value
            nb_neigh=nb_neigh-grid[i][j]
            # if 2 neighbors cell state isn't modified
            if nb_neigh==2:
                cell_state=grid[i][j]
            # if 3 neighbors cell state is alive
            elif nb_neigh==3:
                cell_state=1
            # in other case cell state is dead (default state)
            res_grid[i].append(cell_state)
    return res_grid

Profil de evolution1_corr

In [ ]:
cProfile.run("evolution1_corr(grid)")

## Résultats
On constate une réduction du nombre d'appels de fonction :
- 32000007 pour la 1ère version contre 20000005 pour la 2nde
- en particulier 8000002 appel à la méthode len

# Structures de données
Importation du package numpy et définition de la fonction evolution1_ndarray dans laquelle on utilise des structures de données définies pour le traitement des tableaux : ndarray

In [ ]:
import numpy as np

In [ ]:
def evolution1_ndarray(grid):
    irange=grid.shape[0]
    jrange=grid.shape[1]
    res_grid=np.zeros((irange,jrange))
    for i in range(irange):
        for j in range(jrange):
            # by default cell_state is dead
            cell_state=0
            # loop over neighs to count living cells
            biinf=max(0,i-1)
            bisup=min(irange-1,i+1)
            bjinf=max(0,j-1)
            bjsup=min(jrange-1,j+1)
            nb_neigh=0
            # loop over neighs to count living cells
            for ii in range(biinf,bisup+1):
                for jj in range(bjinf,bjsup+1):
                    if grid[ii][jj] == 1:
                        nb_neigh=nb_neigh+1
            # in my count I count current cell itself substract the value
            nb_neigh=nb_neigh-grid[i][j]
            # if 2 neighbors cell state isn't modified
            if nb_neigh==2:
                cell_state=grid[i][j]
            # if 3 neighbors cell state is alive
            elif nb_neigh==3:
                cell_state=1
            # in other case cell state is dead (default state)
            res_grid[i,j]=cell_state
    return res_grid

Profil de evolution1_ndarray

In [ ]:
cProfile.run("evolution1_ndarray(grid)")

Estimation des temps d'exécution pour chacune des trois fonctions

In [ ]:
%timeit masterjdv.evolution1(grid)

In [ ]:
%timeit evolution1_corr(grid)

In [ ]:
%timeit evolution1_ndarray(grid)